In [74]:
from scipy.spatial import distance as dist
from imutils.video import VideoStream
from imutils import face_utils
from threading import Thread
import numpy as np
from playsound import playsound
import imutils
import time
import dlib
import cv2
import time
import tkinter as tk
from tkinter import filedialog
import winsound
EYE_AR_THRESH = 0.27
MAR_THRESH=0.3


In [75]:
def eye_aspect_ratio(eye):
    # compute the euclidean distances between the two sets of
    # vertical eye landmarks (x, y)-coordinates
    A = dist.euclidean(eye[1], eye[5])
    B = dist.euclidean(eye[2], eye[4])
    # compute the euclidean distance between the horizontal
    # eye landmark (x, y)-coordinates
    C = dist.euclidean(eye[0], eye[3])
    # compute the eye aspect ratio
    ear = (A + B) / (2.0 * C)
    # return the eye aspect ratio
    return ear
def mouth_aspect_ratio(mouth):
    # compute the euclidean distances between the two sets of
    # vertical eye landmarks (x, y)-coordinates
    A = dist.euclidean(mouth[1], mouth[7])
    B = dist.euclidean(mouth[2], mouth[6])
    C = dist.euclidean(mouth[3], mouth[5])
#     D = dist.euclidean(mouth[4], mouth[8])
#     E = dist.euclidean(mouth[5], mouth[7])
    # compute the euclidean distance between the horizontal
    # eye landmark (x, y)-coordinates
    F = dist.euclidean(mouth[0], mouth[4])
    # compute the eye aspect ratio
    mouth = (A + B +C) / (3.0 * F)
    # return the eye aspect ratio
    return mouth


In [76]:

class App:
    def __init__(self, master,EYETHR,MARTHR):
        self.my_var1 = tk.DoubleVar()
        self.my_var2 = tk.DoubleVar()
        self.EYETHR=0
        self.MARTHR=0
        self.my_var1.set(EYETHR)
        self.my_var2.set(MARTHR)
        self.master = master
        master.title("Real Time Drowsiness Detection System")

        self.title_label = tk.Label(master, text="Real Time Drowsiness Detection System", font=('Arial', 20))
        self.title_label.pack(pady=20)

        self.calibrate_button = tk.Button(master, text="Calibrate", font=('Arial', 12), command=self.open_window, padx=20, pady=10)
        self.calibrate_button.pack(pady=20,padx=20)

        self.start_button = tk.Button(master, text="Start", font=('Arial', 12), command=self.start, padx=20, pady=10)
        self.start_button.pack(pady=20,padx=20)
        
        self.start_button = tk.Button(master, text="Guidelines", font=('Arial', 10), command=self.guide, padx=20, pady=10)
        self.start_button.pack(pady=20,padx=20)
        self.var1_label = tk.Label(master, text="Current EAR Threshold:", font=('Arial', 16))
        self.var1_label.pack(pady=10)
        my_label = tk.Label(master, textvariable=str(self.my_var1),font=('Arial', 16))
        my_label.pack()
        self.var1_value = tk.Label(master, text="Current MAR Threshold:", font=('Arial', 16))
        self.var1_value.pack(pady=10)
        my_label = tk.Label(master, textvariable=str(self.my_var2),font=('Arial', 16))
        my_label.pack()
    def guide(self):
            new = tk.Toplevel()
            self.var4_label = tk.Label(new, text="GUIDELINES", font=('Arial', 20))
            self.var4_label.pack(pady=10)
            self.var4_label = tk.Label(new, text="1. Place Your camera slightly above face level.", font=('Arial', 16))
            self.var4_label.pack(pady=20)
            self.var4_label = tk.Label(new, text="2. Adjust Camera angle such that it is facing the face.", font=('Arial', 16))
            self.var4_label.pack(pady=10)
            self.var4_label = tk.Label(new, text="3. Proper lighting conditions will give proper accuracy.", font=('Arial', 16))
            self.var4_label.pack(pady=10)
            self.var4_label = tk.Label(new, text="4. Photos used in calibration should be taken from same orientation as mentioned in 1 and 2.", font=('Arial', 16))
            self.var4_label.pack(pady=10)
    def open_window(self):
    # Create a new window
        new_window = tk.Toplevel()
    
    # Add some content to the window
        label = tk.Label(new_window, text="CALIBRATION",font=('Arial', 20))
        label.pack()
        self.calibrate_button = tk.Button(new_window, text="Upload image with eye open and mouth closed", font=('Arial', 12), command=self.calibrate, padx=20, pady=10)
        self.calibrate_button.pack(pady=20,padx=20)
        self.calibrate_button = tk.Button(new_window, text="Upload image with eye close and mouth open", font=('Arial', 12), command=self.calibrate, padx=20, pady=10)
        self.calibrate_button.pack(pady=20,padx=20)
        self.calibrate_button = tk.Button(new_window, text="OK", font=('Arial', 16), command=self.calculationthreshold, padx=20, pady=10)
        self.calibrate_button.pack(pady=20,padx=20)
    def calculationthreshold(self):
        global EYE_AR_THRESH
        global MAR_THRESH
        self.my_var1.set(self.EYETHR/2)
        self.my_var2.set(self.MARTHR/2)
        EYE_AR_THRESH = self.EYETHR/2
        MAR_THRESH=self.MARTHR/2
        print(EYE_AR_THRESH)
        print(MAR_THRESH)
    def calibrate(self):
        print("Calibrating...")
        lStart=6
        lEnd=11
        rStart=0
        rEnd=5
        file_path = filedialog.askopenfilename(filetypes=[("Image files", "*.jpg;*.jpeg;*.png")])
        # Load the detector
        detector = dlib.get_frontal_face_detector()
        
        # Load the predictor
        predictor = dlib.shape_predictor("../img/predictor.dat")
        
        # Load the image
        image = cv2.imread(file_path)
        
        # Convert the image to grayscale
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        
        # Detect faces in the grayscale image
        faces = detector(gray,0)
        if not faces:
                return 0
        rect=faces[0]
        shape = predictor(gray, rect)
        shape = face_utils.shape_to_np(shape)
        
        leftEye = shape[lStart:(lEnd+1)]
        rightEye = shape[rStart:(rEnd+1)]
        mouth=shape[lEnd+1:(24)]
        internalmouth=shape[24:32]
        mouthMAR=mouth_aspect_ratio(internalmouth)
        print(mouthMAR)
        print(self.MARTHR)
        self.MARTHR=self.MARTHR+mouthMAR
        leftEAR = eye_aspect_ratio(leftEye)
        rightEAR = eye_aspect_ratio(rightEye)
        # average the eye aspect ratio together for both eyes
        print((leftEAR + rightEAR) / 2.0)
        self.EYETHR  = self.EYETHR + (leftEAR + rightEAR) / 2.0
        print(self.EYETHR)
        print(self.MARTHR)
    def start(self):
        print("Starting...")
# define two constants, one for the eye aspect ratio to indicate
# blink and then a second constant for the number of consecutive
# frames the eye must be below the threshold for to set off the
# alarm
        
        EYE_AR_CONSEC_FRAMES = 30
        MOUTH_FRAMES= 30
# initialize the frame counter as well as a boolean used to
# indicate if the alarm is going off
        COUNTER = 0
        counterMouth=0
# initialize dlib's face detector (HOG-based) and then create
# the facial landmark predictor
        print("[INFO] loading facial landmark predictor...")
        detector = dlib.get_frontal_face_detector()
        predictor = dlib.shape_predictor("predictor.dat")
        lStart=6
        lEnd=11
        rStart=0
        rEnd=5
        # start the video stream thread
        print("[INFO] starting video stream thread...")
        vs = VideoStream(src=0).start()
        time.sleep(1.0)
        # used to record the time when we processed last frame
        prev_frame_time = 0
          
        # used to record the time at which we processed current frame
        new_frame_time = 0
        ear=0
        
        # loop over frames from the video stream
        while True:
            
            frame = vs.read()
            frame = imutils.resize(frame, width=600)
            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            rect = detector(gray, 0)
            # Calculating the fps
            new_frame_time = time.time()
                # fps will be number of frame processed in given time frame
                # since their will be most of time error of 0.001 second
                # we will be subtracting it to get more accurate result
            fps = 1/(new_frame_time-prev_frame_time)
            prev_frame_time = new_frame_time
              
                # converting the fps into integer
            fps = int(fps)
                # converting the fps to string so that we can display it on frame
           
            if not rect:
                cv2.putText(frame, "No FACE DETECTED",
                            (200, 40),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
                cv2.putText(frame, "FPS :"+str(fps), (20, 400),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
                cv2.putText(frame, "Press q to exit", (300, 400),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
                
                cv2.imshow("Frame", frame)
                
                
            else:
            
                
                
                rect=rect[0]
                shape = predictor(gray, rect)
                shape = face_utils.shape_to_np(shape)
                # extract the left and right eye coordinates, then use the
                # coordinates to compute the eye aspect ratio for both eyes
                leftEye = shape[lStart:(lEnd+1)]
                rightEye = shape[rStart:(rEnd+1)]
                mouth=shape[lEnd+1:(24)]
                internalmouth=shape[24:32]
                mouthMAR=mouth_aspect_ratio(internalmouth)
                
                leftEAR = eye_aspect_ratio(leftEye)
                rightEAR = eye_aspect_ratio(rightEye)
                # average the eye aspect ratio together for both eyes
                ear = (leftEAR + rightEAR) / 2.0
                # compute the convex hull for the left and right eye, then
                # visualize each of the eyes
                leftEyeHull = cv2.convexHull(leftEye)
                rightEyeHull = cv2.convexHull(rightEye)
                mouthHull=cv2.convexHull(mouth)
                internalmouthHull=cv2.convexHull(internalmouth)
                cv2.drawContours(frame, [leftEyeHull], -1, (0, 255, 0), 1)
                cv2.drawContours(frame, [rightEyeHull], -1, (0, 255, 0), 1)
                cv2.drawContours(frame, [mouthHull], -1, (0, 255, 0), 1)
                cv2.drawContours(frame, [internalmouthHull], -1, (0, 255, 0), 1)
                # check to see if the eye aspect ratio is below the blink
                # threshold, and if so, increment the blink frame counter
                if ear < EYE_AR_THRESH:
                    print(EYE_AR_THRESH)
                    COUNTER += 1
                    # if the eyes were closed for a sufficient number of
                    # then sound the alarm
                    if COUNTER >= fps*2:
                        # if the alarm is not on, turn it on
                        winsound.Beep(1000, 1000) 
                        cv2.putText(frame, "DROWSINESS ALERT!", (250, 20),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
                else:
                    COUNTER = 0
              
                    
                    
                #for MAR
                if mouthMAR > MAR_THRESH:
                    print(MAR_THRESH)
                    counterMouth += 1
                    if counterMouth >=  fps*5:
                        winsound.Beep(1000, 1000) 
                        cv2.putText(frame, "YAWN ALERT!", (250, 20),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
                else:
                    counterMouth = 0
                    # draw the computed eye aspect ratio on the frame to help
                    # with debugging and setting the correct eye aspect ratio
                    # thresholds and frame counters
                cv2.putText(frame, "EAR:{:.2f}".format(ear), (500, 20),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
                cv2.putText(frame, "MAR:{:.2f}".format(mouthMAR), (20, 20),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
                cv2.putText(frame, "FPS :"+str(fps), (20, 400),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
                
                cv2.putText(frame, "Press q to exit", (300, 400),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
                cv2.imshow("Frame", frame)
            key = cv2.waitKey(1) & 0xFF
            
            if key == ord("q"):
                break
        
        cv2.destroyAllWindows()
        vs.stop() 

root = tk.Tk()
app = App(root,EYE_AR_THRESH,MAR_THRESH)
root.mainloop()



Starting...
[INFO] loading facial landmark predictor...
[INFO] starting video stream thread...
0.27
0.27
0.27
0.27
0.27
0.27
0.27
0.27
0.27
0.27
0.27
0.27
0.27
0.27
0.27
0.27
0.27
0.27
0.27
0.27
0.27
0.27
0.27
0.27
0.27
0.27
0.3
0.27
0.3
0.27
0.27
0.3
0.27
0.3
0.27
0.3
0.27
0.27
0.27
0.27
0.27
0.27
0.27
0.27
0.27
0.27
0.27
0.27
0.27
0.27
0.3
0.27
0.27
0.27
0.27
0.27
0.27
0.27
0.27
0.27
0.27
0.27
0.27
0.27
0.27
0.27
0.27
0.27
0.27
0.27
0.27
0.27
0.27
0.27
0.27
0.27
0.27
0.27
0.27
